In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#Read CSV file

city_data = pd.read_csv("Resources/weather_df.csv", encoding= "utf-8")
city_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,bambous virieux,75,MU,1600140355,68,-20.34,57.76,68.0,11.41
1,1,souillac,75,MU,1600140356,68,-20.52,57.52,68.0,11.41
2,2,busselton,0,AU,1600140357,39,-33.65,115.33,73.0,11.23
3,3,cape town,6,ZA,1600140233,81,-33.93,18.42,44.6,1.12
4,4,marsa matruh,13,EG,1600140360,73,31.35,27.25,77.0,7.25


In [3]:
#Add values for HeatMap
locations= city_data[["Lat", "Lng"]]
humidity_weight = city_data["Humidity"]
humidity_max = humidity_weight.max()

In [5]:
#Add HeatMap
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights= humidity_weight, dissipating=False, max_intensity=humidity_max, point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Creating clean DF with specific requirements
clean_df = city_data.loc[(city_data["Max Temp"]>70) & (city_data["Max Temp"]<80) & (city_data["Cloudiness"]==0) &(city_data["Wind Speed"]<10), :]
clean_df=clean_df.dropna()
clean_df = clean_df.drop(["Unnamed: 0"], axis =1)
clean_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
34,bardiyah,0,LY,1600140395,79,31.76,25.09,72.72,3.20
66,san quintin,0,MX,1600140433,63,30.48,-115.95,75.07,8.63
118,geraldton,0,AU,1600140501,57,-28.77,114.60,75.20,8.05
129,victoria point,0,AU,1600140513,61,-27.58,153.30,77.00,5.99
155,sabha,0,LY,1600140544,35,27.04,14.43,76.69,9.10


In [7]:
#Hotel Mapping
hotel_df = clean_df
hotel_df.reset_index(drop= True, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bardiyah,0,LY,1600140395,79,31.76,25.09,72.72,3.20
1,san quintin,0,MX,1600140433,63,30.48,-115.95,75.07,8.63
2,geraldton,0,AU,1600140501,57,-28.77,114.60,75.20,8.05
3,victoria point,0,AU,1600140513,61,-27.58,153.30,77.00,5.99
4,sabha,0,LY,1600140544,35,27.04,14.43,76.69,9.10
5,hun,0,LY,1600140545,70,29.13,15.95,72.28,5.50
6,balkanabat,0,TM,1600140566,44,39.51,54.37,72.00,4.70
7,caravelas,0,BR,1600140614,92,-17.71,-39.25,70.48,5.37
8,gazanjyk,0,TM,1600140671,39,39.24,55.52,74.12,1.86
9,vila velha,0,BR,1600140516,78,-20.33,-40.29,71.60,8.05


In [8]:
#Create a list for the hotels
hotels=[]

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

#get the location for citys
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
#add the location to the parameters
    params["location"]=f'{lat},{lng}'
#set up url and API to request data
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Results for {index}:{row['City']}.")
    response = requests.get(url, params=params).json()
#add results to a variable
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"]= results[0]['name']
    except:
        print("Missing information...skip") 
  

Results for 0:bardiyah.
Closest hotel is Hotel Burdy.
Results for 1:san quintin.
Closest hotel is Hotel Jardines Baja.
Results for 2:geraldton.
Closest hotel is The Gerald Apartment Hotel.
Results for 3:victoria point.
Closest hotel is Oasis on Coochiemudlo.
Results for 4:sabha.
Closest hotel is فندق المواجيد.
Results for 5:hun.
Closest hotel is Alrwasi Hotel.
Results for 6:balkanabat.
Closest hotel is Nebitchi Hotel.
Results for 7:caravelas.
Closest hotel is Pousada Encanto Abrolhos.
Results for 8:gazanjyk.
Missing information...skip
Results for 9:vila velha.
Closest hotel is Hotel Senac Ilha do Boi.
Results for 10:presidente medici.
Closest hotel is Oliveira Palace Hotel.
Results for 11:port said.
Closest hotel is فندق رستا بورسعيد Resta Port Said Hotel.
Results for 12:shepsi.
Closest hotel is Premier Hotel.
Results for 13:jawa.
Closest hotel is Opal Hotel.


In [9]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,bardiyah,0,LY,1600140395,79,31.76,25.09,72.72,3.20,Hotel Burdy
1,san quintin,0,MX,1600140433,63,30.48,-115.95,75.07,8.63,Hotel Jardines Baja
2,geraldton,0,AU,1600140501,57,-28.77,114.60,75.20,8.05,The Gerald Apartment Hotel
3,victoria point,0,AU,1600140513,61,-27.58,153.30,77.00,5.99,Oasis on Coochiemudlo
4,sabha,0,LY,1600140544,35,27.04,14.43,76.69,9.10,فندق المواجيد
5,hun,0,LY,1600140545,70,29.13,15.95,72.28,5.50,Alrwasi Hotel
6,balkanabat,0,TM,1600140566,44,39.51,54.37,72.00,4.70,Nebitchi Hotel
7,caravelas,0,BR,1600140614,92,-17.71,-39.25,70.48,5.37,Pousada Encanto Abrolhos
8,gazanjyk,0,TM,1600140671,39,39.24,55.52,74.12,1.86,NaN
9,vila velha,0,BR,1600140516,78,-20.33,-40.29,71.60,8.05,Hotel Senac Ilha do Boi


In [10]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))